In [ ]:
import os
import pandas as pd

def encontrar_raiz_proyecto(archivo_raiz="README.md") -> str:
    """Encuentra la raíz del proyecto buscando un archivo identificador"""
    ruta = os.path.abspath(os.getcwd())
    while True:
        if archivo_raiz in os.listdir(ruta):
            return ruta
        nueva_ruta = os.path.dirname(ruta)
        if nueva_ruta == ruta:  # Llegamos a la raíz del sistema sin encontrar
            raise FileNotFoundError(f"No se encontró {archivo_raiz} en ningún directorio padre")
        ruta = nueva_ruta

def leer_tsv_chunks(ruta: str):
    return pd.read_csv(ruta, sep='\t', chunksize=10000)

In [18]:
titulos_iterable = leer_tsv_chunks(RUTA_TITULO)

# Conjunto para almacenar tipos únicos
tipos_unicos = set()

# Leer chunk por chunk y añadir los tipos únicos al conjunto
for chunk in titulos_iterable:
    tipos_unicos.update(chunk['titleType'].unique())

# Convertir a lista y ordenar alfabéticamente
lista_tipos = sorted(list(tipos_unicos))
print("Tipos de títulos únicos encontrados:")
for tipo in lista_tipos:
    print(f"- {tipo}")

Tipos de títulos únicos encontrados:
- movie
- short
- tvEpisode
- tvMiniSeries
- tvMovie
- tvPilot
- tvSeries
- tvShort
- tvSpecial
- video
- videoGame


In [19]:
# Lista de tipos que queremos filtrar
tipos_buscados = ['short', 'tvMovie', 'tvShort', 'tvSpecial']

# Diccionario para almacenar los resultados de cada tipo
resultados = {tipo: [] for tipo in tipos_buscados}

# Leer chunks y filtrar
titulos_iterable = leer_tsv_chunks(RUTA_TITULO)

for chunk in titulos_iterable:
    for tipo in tipos_buscados:
        # Si aún no tenemos 5 registros de este tipo, buscamos más
        if len(resultados[tipo]) < 5:
            filtrados = chunk[chunk['titleType'] == tipo]
            # Tomamos solo los que faltan hasta llegar a 5
            resultados[tipo].extend(filtrados.head(5 - len(resultados[tipo])).to_dict('records'))
    
    # Verificar si ya tenemos 5 de cada tipo
    if all(len(registros) >= 5 for registros in resultados.values()):
        break

# Mostrar resultados
for tipo in tipos_buscados:
    print(f"\n=== {tipo.upper()} ===")
    print("Título Original | Año Inicio | Duración | Géneros")
    print("-" * 70)
    for registro in resultados[tipo]:
        print(f"{registro['originalTitle'][:30]:<30} | {registro['startYear']:<10} | {registro['runtimeMinutes']:<8} | {registro['genres']}")


=== SHORT ===
Título Original | Año Inicio | Duración | Géneros
----------------------------------------------------------------------
Carmencita                     | 1894       | 1        | Documentary,Short
Le clown et ses chiens         | 1892       | 5        | Animation,Short
Pauvre Pierrot                 | 1892       | 5        | Animation,Comedy,Romance
Un bon bock                    | 1892       | 12       | Animation,Short
Blacksmith Scene               | 1893       | 1        | Short

=== TVMOVIE ===
Título Original | Año Inicio | Duración | Géneros
----------------------------------------------------------------------
Julius Caesar                  | 1938       | 101      | Drama,History
As You Like It                 | 1946       | \N       | Drama
A Midsummer Night's Dream      | 1946       | 150      | Drama,Fantasy
Hamlet Part 1                  | 1947       | 88       | Drama
The Merchant of Venice         | 1947       | 90       | Drama

=== TVSHORT ===
Título Origi

In [20]:
# Tipos de películas que queremos mantener
tipos_peliculas = ['movie', 'tvMovie']

# Lista para ir acumulando los chunks filtrados
peliculas_filtradas = []

# Leer y filtrar por chunks
titulos_iterable = leer_tsv_chunks(RUTA_TITULO)
for chunk in titulos_iterable:
    # Filtrar solo los tipos que queremos
    chunk_filtrado = chunk[chunk['titleType'].isin(tipos_peliculas)]
    peliculas_filtradas.append(chunk_filtrado)

# Concatenar todos los chunks filtrados en un único DataFrame
df_peliculas = pd.concat(peliculas_filtradas, ignore_index=True)

# Mostrar información general
print(f"Total de registros filtrados: {len(df_peliculas)}")
print("\nDistribución por tipo:")
print(df_peliculas['titleType'].value_counts())

print("\nPrimeras 5 películas de cada tipo:")
for tipo in tipos_peliculas:
    print(f"\n=== {tipo.upper()} ===")
    print(df_peliculas[df_peliculas['titleType'] == tipo].head()[['primaryTitle', 'startYear', 'runtimeMinutes', 'genres']])

Total de registros filtrados: 876594

Distribución por tipo:
titleType
movie      724491
tvMovie    152103
Name: count, dtype: int64

Primeras 5 películas de cada tipo:

=== MOVIE ===
                    primaryTitle startYear runtimeMinutes  \
0                     Miss Jerry      1894             45   
1  The Corbett-Fitzsimmons Fight      1897            100   
2                       Bohemios      1905            100   
3    The Story of the Kelly Gang      1906             70   
4               The Prodigal Son      1907             90   

                       genres  
0                     Romance  
1      Documentary,News,Sport  
2                          \N  
3  Action,Adventure,Biography  
4                       Drama  

=== TVMOVIE ===
                    primaryTitle startYear runtimeMinutes         genres
20302              Julius Caesar      1938            101  Drama,History
26711             As You Like It      1946             \N          Drama
27062  A Midsummer Ni

In [21]:
# Función para contar valores nulos y '\N'
def contar_nulos(df, columna):
    nulos_N = df[df[columna] == '\\N'].shape[0]  # Contar '\N'
    nulos_nan = df[columna].isna().sum()         # Contar NaN
    total = nulos_N + nulos_nan
    porcentaje = (total / len(df)) * 100
    return {
        'Valores \\N': nulos_N,
        'Valores NaN': nulos_nan,
        'Total Nulos': total,
        'Porcentaje': f"{porcentaje:.2f}%"
    }

# Analizar cada columna
print("Análisis de valores nulos por columna:")
print("-" * 50)

for columna in df_peliculas.columns:
    resultados = contar_nulos(df_peliculas, columna)
    print(f"\nColumna: {columna}")
    print(f"- Valores '\\N': {resultados['Valores \\N']:,}")
    print(f"- Valores NaN: {resultados['Valores NaN']:,}")
    print(f"- Total nulos: {resultados['Total Nulos']:,}")
    print(f"- Porcentaje: {resultados['Porcentaje']}")

# Mostrar también cuántos registros tienen al menos un valor nulo
registros_con_nulos = df_peliculas[df_peliculas.apply(lambda x: (x == '\\N').any() or x.isna().any(), axis=1)]
print(f"\nRegistros con al menos un valor nulo: {len(registros_con_nulos):,} ({(len(registros_con_nulos)/len(df_peliculas)*100):.2f}% del total)")

Análisis de valores nulos por columna:
--------------------------------------------------

Columna: tconst
- Valores '\N': 0
- Valores NaN: 0
- Total nulos: 0
- Porcentaje: 0.00%

Columna: titleType
- Valores '\N': 0
- Valores NaN: 0
- Total nulos: 0
- Porcentaje: 0.00%

Columna: titleType
- Valores '\N': 0
- Valores NaN: 0
- Total nulos: 0
- Porcentaje: 0.00%

Columna: primaryTitle
- Valores '\N': 0
- Valores NaN: 3
- Total nulos: 3
- Porcentaje: 0.00%

Columna: primaryTitle
- Valores '\N': 0
- Valores NaN: 3
- Total nulos: 3
- Porcentaje: 0.00%

Columna: originalTitle
- Valores '\N': 0
- Valores NaN: 3
- Total nulos: 3
- Porcentaje: 0.00%

Columna: isAdult
- Valores '\N': 0
- Valores NaN: 0
- Total nulos: 0
- Porcentaje: 0.00%

Columna: startYear
- Valores '\N': 111,390
- Valores NaN: 0
- Total nulos: 111,390
- Porcentaje: 12.71%

Columna: originalTitle
- Valores '\N': 0
- Valores NaN: 3
- Total nulos: 3
- Porcentaje: 0.00%

Columna: isAdult
- Valores '\N': 0
- Valores NaN: 0
- Total

In [22]:
# Columnas a verificar
columnas_a_filtrar = ['genres', 'primaryTitle', 'originalTitle', 'runtimeMinutes']

# Crear una máscara para filtrar
mascara = True  # Comenzamos con todos los registros
for columna in columnas_a_filtrar:
    # Filtrar tanto '\N' como NaN
    mascara = mascara & (df_peliculas[columna] != '\\N') & (~df_peliculas[columna].isna())

# Aplicar el filtro
df_peliculas_limpio = df_peliculas[mascara]

# Mostrar información sobre los registros filtrados
registros_eliminados = len(df_peliculas) - len(df_peliculas_limpio)
print(f"Registros originales: {len(df_peliculas):,}")
print(f"Registros después de eliminar nulos: {len(df_peliculas_limpio):,}")
print(f"Registros eliminados: {registros_eliminados:,} ({(registros_eliminados/len(df_peliculas)*100):.2f}%)")

print("\nDistribución por tipo después de la limpieza:")
print(df_peliculas_limpio['titleType'].value_counts())

print("\nMuestra de 5 registros limpios:")
df_peliculas_limpio[['primaryTitle', 'titleType', 'startYear', 'runtimeMinutes', 'genres']].head()

Registros originales: 876,594
Registros después de eliminar nulos: 530,367
Registros eliminados: 346,227 (39.50%)

Distribución por tipo después de la limpieza:
titleType
movie      431138
tvMovie     99229
Name: count, dtype: int64

Muestra de 5 registros limpios:


,primaryTitle,titleType,startYear,runtimeMinutes,genres
0,Miss Jerry,movie,1894,45,Romance
1,The Corbett-Fitzsimmons Fight,movie,1897,100,"Documentary,News,Sport"
3,The Story of the Kelly Gang,movie,1906,70,"Action,Adventure,Biography"
4,The Prodigal Son,movie,1907,90,Drama
8,The Fairylogue and Radio-Plays,movie,1908,120,"Adventure,Fantasy"


In [23]:
# Filtrar películas del 2019
df_peliculas_filtrado = df_peliculas_limpio[df_peliculas_limpio['startYear'] == '2019']

print(f"Total de películas del 2019: {len(df_peliculas_filtrado):,}")
print("\nDistribución por tipo:")
print(df_peliculas_filtrado['titleType'].value_counts())

print("\nEstadísticas de duración (en minutos):")
print(df_peliculas_filtrado['runtimeMinutes'].astype(float).describe())

print("\nMuestra de películas del 2019:")
df_peliculas_filtrado.head(10)

ids_peliculas = df_peliculas_filtrado['tconst']



Total de películas del 2019: 16,833

Distribución por tipo:
titleType
movie      14637
tvMovie     2196
Name: count, dtype: int64

Estadísticas de duración (en minutos):
count    16833.000000
mean        93.571021
std        514.616846
min          1.000000
25%         70.000000
50%         88.000000
75%        101.000000
max      59460.000000
Name: runtimeMinutes, dtype: float64

Muestra de películas del 2019:


In [ ]:
# Función para filtrar chunks basado en los IDs
def filtrar_por_ids(chunk):
    return chunk[chunk['tconst'].isin(ids_peliculas)]

# Filtrar tabla de críticas
print("Filtrando críticas...")
chunks_filtrados = []
criticas_iterable = leer_tsv_chunks(RUTA_CRITICAS)
for chunk in criticas_iterable:
    filtrado = filtrar_por_ids(chunk)
    if not filtrado.empty:
        chunks_filtrados.append(filtrado)
df_criticas_filtrado = pd.concat(chunks_filtrados, ignore_index=True) if chunks_filtrados else pd.DataFrame()


# Filtrar tabla de personal
print("Filtrando personal...")
chunks_filtrados = []
personal_iterable = leer_tsv_chunks(RUTA_PERSONAL)
for chunk in personal_iterable:
    filtrado = filtrar_por_ids(chunk)
    if not filtrado.empty:
        chunks_filtrados.append(filtrado)
df_personal_filtrado = pd.concat(chunks_filtrados, ignore_index=True) if chunks_filtrados else pd.DataFrame()

# Filtrar tabla de principales
print("Filtrando principales...")
chunks_filtrados = []
principales_iterable = leer_tsv_chunks(RUTA_PRINCIPALES)
for chunk in principales_iterable:
    filtrado = filtrar_por_ids(chunk)
    if not filtrado.empty:
        chunks_filtrados.append(filtrado)
df_principales_filtrado = pd.concat(chunks_filtrados, ignore_index=True) if chunks_filtrados else pd.DataFrame()

# Filtrar tabla de nombres
print("Filtrando nombres...")
chunks_filtrados = []
nombres_iterable = leer_tsv_chunks(RUTA_NOMBRE)
for chunk in nombres_iterable:
    filtrado = filtrar_por_ids(chunk)
    if not filtrado.empty:
        chunks_filtrados.append(filtrado)
df_nombres_filtrado = pd.concat(chunks_filtrados, ignore_index=True) if chunks_filtrados else pd.DataFrame()

Filtrando críticas...
Filtrando nombres...
Filtrando nombres...


KeyError: 'tconst'

In [ ]:
# Guardar como CSV
ruta_salida = os.path.join(DIR_RAIZ, "data", "procesado")
os.makedirs(ruta_salida, exist_ok=True)

df_peliculas_filtrado.to_csv(os.path.join(ruta_salida, "peliculas_2019.csv"), index=False)
df_nombres_filtrado.to_csv(os.path.join(ruta_salida, "nombres_2019.csv"), index=False)
df_principales_filtrado.to_csv(os.path.join(ruta_salida, "principales_2019.csv"), index=False)
df_criticas_filtrado.to_csv(os.path.join(ruta_salida, "criticas_2019.csv"), index=False)
df_personal_filtrado.to_csv(os.path.join(ruta_salida, "personal_2019.csv"), index=False)

print("\nArchivos guardados en la carpeta 'data/procesado/'")
print("Muestra de los datos:")
print("\nPelículas:")
print(df_peliculas_filtrado.head())
print("\nCríticas:")
print(df_criticas_filtrado.head())
print("\nPersonal:")
print(df_personal_filtrado.head())
print("\nNombres:")
print(df_nombres_filtrado.head())
print("\nPrincipales:")
print(df_principales_filtrado.head())

In [31]:
# Conjunto para almacenar profesiones únicas
profesiones_unicas = set()

# Leer chunk por chunk
nombres_iterable = leer_tsv_chunks(RUTA_NOMBRE)
for chunk in nombres_iterable:
    # Filtrar filas donde primaryProfession no sea nulo o '\N'
    profesiones_validas = chunk[~chunk['primaryProfession'].isna() & (chunk['primaryProfession'] != '\\N')]
    
    # Para cada fila, dividir las profesiones y añadirlas al conjunto
    for profesiones in profesiones_validas['primaryProfession']:
        profesiones_unicas.update(profesiones.split(','))

# Convertir a lista y ordenar alfabéticamente
lista_profesiones = sorted(list(profesiones_unicas))
print(f"Total de profesiones únicas encontradas: {len(lista_profesiones)}")
print("\nLista de profesiones:")
for profesion in lista_profesiones:
    print(f"- {profesion}")

Total de profesiones únicas encontradas: 46

Lista de profesiones:
- accountant
- actor
- actress
- animation_department
- archive_footage
- archive_sound
- art_department
- art_director
- assistant
- assistant_director
- camera_department
- casting_department
- casting_director
- choreographer
- cinematographer
- composer
- costume_department
- costume_designer
- director
- editor
- editorial_department
- electrical_department
- executive
- legal
- location_management
- make_up_department
- manager
- miscellaneous
- music_artist
- music_department
- podcaster
- producer
- production_department
- production_designer
- production_manager
- publicist
- script_department
- set_decorator
- sound_department
- soundtrack
- special_effects
- stunts
- talent_agent
- transportation_department
- visual_effects
- writer


In [27]:
principales_iterable = leer_tsv_chunks(RUTA_PRINCIPALES)

ultimo_chunk = None
for chunk in principales_iterable:
    ultimo_chunk = chunk
    
ultimo_chunk.tail(10)

,tconst,ordering,nconst,category,job,characters
94444251,tt9916880,12,nm2676923,actress,\N,"[""Sour Susan""]"
94444252,tt9916880,13,nm2676923,actress,\N,"[""Goody-Goody Gordon""]"
94444253,tt9916880,14,nm2676923,actress,\N,"[""Singing Soraya""]"
94444254,tt9916880,15,nm1469295,actress,\N,"[""Perfect Peter""]"
94444255,tt9916880,16,nm1469295,actress,\N,"[""Lazy Linda""]"
94444256,tt9916880,17,nm0996406,director,principal director,\N
94444257,tt9916880,18,nm1482639,writer,\N,\N
94444258,tt9916880,19,nm2586970,writer,books,\N
94444259,tt9916880,20,nm1594058,producer,producer,\N
94444260,tt9916880,21,nm1482639,producer,producer,\N


In [28]:
criticas_iterable = leer_tsv_chunks(RUTA_CRITICAS)

ultimo_chunk = None
for chunk in criticas_iterable:
    ultimo_chunk = chunk
    
ultimo_chunk.tail(10)

,tconst,averageRating,numVotes
1607719,tt9916836,5.8,6
1607720,tt9916838,5.8,6
1607721,tt9916840,6.8,12
1607722,tt9916842,5.6,6
1607723,tt9916844,5.6,6
1607724,tt9916846,5.8,6
1607725,tt9916848,5.8,6
1607726,tt9916850,5.8,6
1607727,tt9916852,5.8,6
1607728,tt9916880,7.7,10


In [13]:
def direccionArchivo():
    absFilePath=os.path.abspath(__file__)
    path,filename=os.path.split(absFilePath)

    return path

def crearData():
    path=direccionArchivo()
    return pd.read_csv(path+"./mundial.csv")